# 10 - Pretraining CamemBERT

Run the script below with :
- train_file: the local absolute path to paper-ner-bench-das22/dataset/unsupervised_pretraining/10-normalized/all.txt
- output_dir: where to save the pretrained model

## Create the filtered pre-training set

In [35]:
import nltk

# Filter the normalized pre-training 
# Keep only entries containing at least MIN_WORDS_PER_ENTRY words
PRETRAINING_SET_PATH = Path("/home/bertrand/dev/paper-ner-bench-das22/dataset/unsupervised_pretraining/10-normalized/all.txt")
MIN_WORDS_PER_ENTRY = 7 # Keep simples entries like "Morel abbé ,Tournon, 14."

nltk.download("punkt")
nlines = 0
valid_entries = []

with open(PRETRAINING_SET_PATH, "r", encoding="utf-8") as fp:
    for l in fp:
        l = l.strip() # Sanitize
        words = nltk.word_tokenize(l, language="fr", preserve_line=True)
        if len(words) >= MIN_WORDS_PER_ENTRY:
            valid_entries.append(l)
        nlines += 1
        
with open(PRETRAINING_SET_PATH.parent / "all_filtered.txt", "w") as wfp:
    wfp.write("\n".join(valid_entries))

print("Valid entries:%d, %f.2 percent of the total" % (len(valid_entries),len(valid_entries)/nlines))

[nltk_data] Downloading package punkt to /home/bertrand/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Valid entries:845014, 0.808105.2 percent of the total


## Pretrain CamemBERT !

In [33]:
!python pretrain_camembert.py --model_name_or_path "Jean-Baptiste/camembert-ner" --train_file "/home/bertrand/dev/paper-ner-bench-das22/dataset/unsupervised_pretraining/10-normalized/all_filtered.txt" --do_train --do_eval --line_by_line --output_dir "./10-camembert_pretraining/model" --save_total_limit 1 --load_best_model_at_end True --evaluation_strategy "steps" --save_strategy "steps"

01/16/2022 00:43:49 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
01/16/2022 00:43:49 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_s